In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Imposta le tue credenziali Spotify
client_id = '93193686a98745eda05ccb0c72ec7fee'
client_secret = '8eb7dd68e0f346a08abfa9ba41a6d12a'

# Crea un oggetto Spotipy
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Cerca gli album dei Beatles
results = sp.search(q='artist:The Beatles', type='album', limit=5)

# Estrai le informazioni sugli album e scarica le copertine
for album in results['albums']['items']:
    album_title = album['name']
    cover_url = album['images'][0]['url'] if album['images'] else None

    # Scarica la copertina o gestisci come preferisci
    if cover_url:
        # Aggiungi qui la logica per scaricare o gestire le copertine
        print(f"Album: {album_title}, Cover URL: {cover_url}")
    else:
        print(f"Album: {album_title}, Nessuna copertina disponibile")


In [59]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

# Imposta le tue credenziali Spotify
client_id = '93193686a98745eda05ccb0c72ec7fee'
client_secret = '8eb7dd68e0f346a08abfa9ba41a6d12a'

# Crea un oggetto Spotipy
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Leggi il file di testo e estrai solo i nomi degli artisti 
with open('/Users/danielnotaristefano/Desktop/Proggetto data mining/lastfm/merged_df.txt', 'r') as file:
    artists_list = [line.strip().split('\t') for line in file]

# Lista per salvare gli URL delle copertine degli album
cover_urls = []

# Carica le copertine già scaricate
if os.path.exists('cover_urls.txt'):
    with open('cover_urls.txt', 'r') as url_file:
        cover_urls = [line.strip() for line in url_file]

# Creazione della cartella 'copertine'
os.makedirs('copertine', exist_ok=True)

# Scarica le copertine degli album per ciascun artista
for artist_info in artists_list:
    artist_name = artist_info[1]
    

    # Verifica se l'artista è già stato elaborato
    if any(artist_name in url for url in cover_urls):
        print(f"Artista già elaborato: {artist_name}")
        continue

    # Cerca gli album dell'artista su Spotify
    try:
        results = sp.search(q=f'artist:{artist_name}', type='album', limit=5)
    except requests.exceptions.Timeout:
        print(f"Timeout durante la ricerca su Spotify per l'artista: {artist_name}")
        continue

    # Scarica le copertine degli album
    for album in results['albums']['items']:
        album_title = album['name']
        cover_url = album['images'][0]['url'] if album['images'] else None

        if cover_url:
            # Aggiungi l'URL alle copertine alla lista
            cover_urls.append(f"{artist_name}\t{album_title}\t{cover_url}")

            # Scarica la copertina
            response = requests.get(cover_url)

            # Salva la copertina nella cartella 'copertine'
            file_path = os.path.join('copertine', f"{artist_name.replace('/', '_')}_{album_title.replace('/', '_')}_cover.jpg")
            with open(file_path, 'wb') as img_file:
                img_file.write(response.content)

            print(f"Artista: {artist_name}, Album: {album_title}, Copertina scaricata")
        else:
            print(f"Artista: {artist_name}, Album: {album_title}, Nessuna copertina disponibile")

# Salva gli URL delle copertine in un file di testo
with open('cover_urls.txt', 'w') as url_file:
    for url in cover_urls:
        url_file.write(url + '\n')


Artista: Diary of Dreams, Album: Grau im Licht, Copertina scaricata
Artista: Diary of Dreams, Album: The Anatomy of Silence, Copertina scaricata
Artista: Diary of Dreams, Album: Hell in Eden, Copertina scaricata
Artista: Diary of Dreams, Album: Melancholin, Copertina scaricata
Artista: Diary of Dreams, Album: Freak Perfume, Copertina scaricata
Artista: Moonspell, Album: Wolfheart, Copertina scaricata
Artista: Moonspell, Album: Night Eternal (Extended Version), Copertina scaricata
Artista: Moonspell, Album: Irreligious, Copertina scaricata
Artista: Moonspell, Album: Extinct (Deluxe Version), Copertina scaricata
Artista: Moonspell, Album: Memorial (Bonus Track Edition), Copertina scaricata
Artista: Marilyn Manson, Album: Antichrist Superstar, Copertina scaricata
Artista: Marilyn Manson, Album: Smells Like Children, Copertina scaricata
Artista: Marilyn Manson, Album: Holy Wood, Copertina scaricata
Artista: Marilyn Manson, Album: Lest We Forget - The Best Of, Copertina scaricata
Artista: M

In [ ]:
!pip install youtube_dl
!pip install yt_dlp

In [60]:
import os
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from youtubesearchpython import VideosSearch
import yt_dlp

# Imposta le tue credenziali Spotify
client_id = '93193686a98745eda05ccb0c72ec7fee'
client_secret = '8eb7dd68e0f346a08abfa9ba41a6d12a'

# Crea un oggetto Spotipy
sp = Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Cartella di destinazione
output_folder = 'tracce'
output_folder_links = 'tracce_link'

# Configura yt_dlp
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '128',
    }],
    'outtmpl': os.path.join(output_folder, "%(title)s.%(ext)s"),
}

# Leggi il file di testo e estrai solo i nomi degli artisti
with open('/Users/danielnotaristefano/Desktop/Proggetto data mining/lastfm/merged_df.txt', 'r') as file:
    artists_list = [line.strip().split('\t')[1] for line in file]

# Scarica le tracce audio per ciascun artista
for artist_name in artists_list:
    # Cerca le tracce dell'artista su Spotify
    results = sp.search(q=f'artist:{artist_name}', type='track', limit=5)

    # Cerca e scarica le tracce su diverse piattaforme
    for track in results['tracks']['items']:
        track_name = track['name']
        track_id = track['id']

        # Cerca fino a 5 link su diverse piattaforme
        platforms = ['youtube', 'soundcloud', 'vimeo']  # Esempi di piattaforme
        for platform in platforms:
            video_search = VideosSearch(f"{artist_name} {track_name} {platform}", limit=1)
            results_video = video_search.result()

            try:
                if results_video['result']:
                    video_url = results_video['result'][0]['link']

                    # Salva il link della piattaforma
                    with open(os.path.join(output_folder_links, f"{artist_name}_{track_name}_{platform}_link.txt"), 'w') as link_file:
                        link_file.write(video_url)

                    # Utilizza yt_dlp per scaricare l'audio dalla piattaforma
                    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                        ydl.download([video_url])
                        print(f"Artista: {artist_name}, Traccia: {track_name}, Audio scaricato da {platform}")
                        break  # Esce dal loop se il download è riuscito
            except Exception as e:
                print(f"Errore durante il tentativo su {platform} per Artista: {artist_name}, Traccia: {track_name}: {e}")
        else:
            print(f"Nessun link funzionante trovato per Artista: {artist_name}, Traccia: {track_name}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=6WD2_Nhc2I0
[youtube] 6WD2_Nhc2I0: Downloading webpage
[youtube] 6WD2_Nhc2I0: Downloading ios player API JSON
[youtube] 6WD2_Nhc2I0: Downloading android player API JSON
[youtube] 6WD2_Nhc2I0: Downloading m3u8 information
[info] 6WD2_Nhc2I0: Downloading 1 format(s): 251
[download] Destination: tracce/Diary Of Dreams - She And Her Darkness [HQ].webm
[download] 100% of    5.15MiB in 00:00:00 at 8.23MiB/s   
[ExtractAudio] Destination: tracce/Diary Of Dreams - She And Her Darkness [HQ].mp3
Deleting original file tracce/Diary Of Dreams - She And Her Darkness [HQ].webm (pass -k to keep)
Artista: Diary of Dreams, Traccia: She and Her Darkness, Audio scaricato da youtube
[youtube] Extracting URL: https://www.youtube.com/watch?v=dlSWykvqM6Y
[youtube] dlSWykvqM6Y: Downloading webpage
[youtube] dlSWykvqM6Y: Downloading ios player API JSON
[youtube] dlSWykvqM6Y: Downloading android player API JSON
[youtube] dlSWykvqM6Y: Downloading m3u8 inf

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [69]:
import os
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from youtubesearchpython import VideosSearch
import yt_dlp

# Imposta le tue credenziali Spotify
client_id = '93193686a98745eda05ccb0c72ec7fee'
client_secret = '8eb7dd68e0f346a08abfa9ba41a6d12a'

# Crea un oggetto Spotipy
# Crea un oggetto Spotipy con un timeout maggiore
sp = Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret, requests_timeout=20))


# Cartella di destinazione
output_folder = 'tracce'
output_folder_links = 'tracce_link'

# Configura yt_dlp
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'outtmpl': os.path.join(output_folder, "%(title)s.%(ext)s"),
}

# Leggi il file di testo e estrai solo i nomi degli artisti
with open('/Users/danielnotaristefano/Desktop/Proggetto data mining/lastfm/merged_df.txt', 'r') as file:
    artists_list = [line.strip().split('\t')[1] for line in file]

# Scarica le tracce audio per ciascun artista
for artist_name in artists_list:
    # Cerca le tracce dell'artista su Spotify
    try:
        results = sp.search(q=f'artist:{artist_name}', type='track', limit=5)
    except Exception as e:
        print(f"Errore durante la ricerca di {artist_name} su Spotify: {e}")
        continue  # Passa al prossimo artista in caso di errore

    # Cerca e scarica le tracce su diverse piattaforme
    for track in results['tracks']['items']:
        track_name = track['name']
        track_id = track['id']

        # Cerca fino a 5 link su diverse piattaforme
        platforms = ['youtube', 'soundcloud', 'vimeo']  # Esempi di piattaforme
        for platform in platforms:
            try:
                video_search = VideosSearch(f"{artist_name} {track_name} {platform}", limit=1)
                results_video = video_search.result()

                if results_video['result']:
                    video_url = results_video['result'][0]['link']

                    # Salva il link della piattaforma
                    with open(os.path.join(output_folder_links, f"{artist_name}_{track_name}_{platform}_link.txt"), 'w') as link_file:
                        link_file.write(video_url)

                    # Utilizza yt_dlp per scaricare l'audio dalla piattaforma
                    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                        try:
                            ydl.download([video_url])
                            print(f"Artista: {artist_name}, Traccia: {track_name}, Audio scaricato da {platform}")
                            break  # Esce dal loop se il download è riuscito
                        except yt_dlp.utils.DownloadError as e:
                            print(f"Tentativo su {platform} fallito per Artista: {artist_name}, Traccia: {track_name}: {e}")
            except Exception as e:
                print(f"Errore durante la ricerca di {artist_name} {track_name} su {platform}: {e}")
                continue  # Passa alla prossima piattaforma in caso di errore
        else:
            print(f"Artista: {artist_name}, Traccia: {track_name}, Nessun risultato su nessuna piattaforma")

# Se tutti i tentativi falliscono, passa alla prossima traccia
else:
    print(f"Nessun link funzionante trovato per Artista: {artist_name}, Traccia: {track_name}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=6WD2_Nhc2I0
[youtube] 6WD2_Nhc2I0: Downloading webpage
[youtube] 6WD2_Nhc2I0: Downloading ios player API JSON
[youtube] 6WD2_Nhc2I0: Downloading android player API JSON
[youtube] 6WD2_Nhc2I0: Downloading m3u8 information
[info] 6WD2_Nhc2I0: Downloading 1 format(s): 251
[download] Destination: tracce/Diary Of Dreams - She And Her Darkness [HQ].webm
[download] 100% of    5.15MiB in 00:00:00 at 16.34MiB/s  
[ExtractAudio] Destination: tracce/Diary Of Dreams - She And Her Darkness [HQ].mp3
Deleting original file tracce/Diary Of Dreams - She And Her Darkness [HQ].webm (pass -k to keep)
Artista: Diary of Dreams, Traccia: She and Her Darkness, Audio scaricato da youtube
[youtube] Extracting URL: https://www.youtube.com/watch?v=dlSWykvqM6Y
[youtube] dlSWykvqM6Y: Downloading webpage
[youtube] dlSWykvqM6Y: Downloading ios player API JSON
[youtube] dlSWykvqM6Y: Downloading android player API JSON
[youtube] dlSWykvqM6Y: Downloading m3u8 inf

[download] Got error: HTTPSConnectionPool(host='rr5---sn-hpa7knl7.googlevideo.com', port=443): Read timed out.


[download] Got error: HTTPSConnectionPool(host='rr5---sn-hpa7knl7.googlevideo.com', port=443): Read timed out.
[youtube] Extracting URL: https://www.youtube.com/watch?v=2z5Po1-s0FQ
[youtube] 2z5Po1-s0FQ: Downloading webpage
[youtube] 2z5Po1-s0FQ: Downloading ios player API JSON
[youtube] 2z5Po1-s0FQ: Downloading android player API JSON
[youtube] 2z5Po1-s0FQ: Downloading m3u8 information


[info] 2z5Po1-s0FQ: Downloading 1 format(s): 251
[download] Destination: tracce/The All American Rejects - It Ends Tonight (Doomer).webm
[download] 100% of    4.26MiB in 00:00:01 at 2.96MiB/s   
[ExtractAudio] Destination: tracce/The All American Rejects - It Ends Tonight (Doomer).mp3
Deleting original file tracce/The All American Rejects - It Ends Tonight (Doomer).webm (pass -k to keep)
Artista: The All-American Rejects, Traccia: It Ends Tonight, Audio scaricato da soundcloud
Errore durante la ricerca di The Rasmus su Spotify: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)
[youtube] Extracting URL: https://www.youtube.com/watch?v=ZBL5y8AciNw
[youtube] ZBL5y8AciNw: Downloading webpage
[youtube] ZBL5y8AciNw: Downloading ios player API JSON
[youtube] ZBL5y8AciNw: Downloading android player API JSON
[youtube] ZBL5y8AciNw: Downloading m3u8 information
[info] ZBL5y8AciNw: Downloading 1 format(s): 251
[download] Destination: tracce/The Day Of Night.we

[download] Got error: HTTPSConnectionPool(host='rr2---sn-hpa7znzy.googlevideo.com', port=443): Read timed out. (read timeout=20.0)


[download] Got error: HTTPSConnectionPool(host='rr2---sn-hpa7znzy.googlevideo.com', port=443): Read timed out. (read timeout=20.0)
[youtube] Extracting URL: https://www.youtube.com/watch?v=7bg4Dy6jT_c
[youtube] 7bg4Dy6jT_c: Downloading webpage
[youtube] 7bg4Dy6jT_c: Downloading ios player API JSON
[youtube] 7bg4Dy6jT_c: Downloading android player API JSON
[youtube] 7bg4Dy6jT_c: Downloading m3u8 information
[info] 7bg4Dy6jT_c: Downloading 1 format(s): 251
[download] Destination: tracce/No Giving Up (MAD ZERØ x UPSKIBËÂT).webm
[download] 100% of    2.55MiB in 00:00:01 at 2.34MiB/s   
[ExtractAudio] Destination: tracce/No Giving Up (MAD ZERØ x UPSKIBËÂT).mp3
Deleting original file tracce/No Giving Up (MAD ZERØ x UPSKIBËÂT).webm (pass -k to keep)
Artista: Crossfade, Traccia: No Giving Up, Audio scaricato da soundcloud
[youtube] Extracting URL: https://www.youtube.com/watch?v=4rh8d5w-wDE
[youtube] 4rh8d5w-wDE: Downloading webpage
[youtube] 4rh8d5w-wDE: Downloading ios player API JSON
[youtu

[info] ItIEXbqzhGg: Downloading 1 format(s): 251
[download] Destination: tracce/The Execution Of All Things.webm
[download] 100% of    4.05MiB in 00:00:25 at 161.13KiB/s 
[ExtractAudio] Destination: tracce/The Execution Of All Things.mp3
Deleting original file tracce/The Execution Of All Things.webm (pass -k to keep)
Artista: Rilo Kiley, Traccia: The Execution Of All Things, Audio scaricato da youtube
Errore durante la ricerca di The Cribs su Spotify: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)
[youtube] Extracting URL: https://www.youtube.com/watch?v=CqM11bt9QvI
[youtube] CqM11bt9QvI: Downloading webpage
[youtube] CqM11bt9QvI: Downloading ios player API JSON
[youtube] CqM11bt9QvI: Downloading android player API JSON
[youtube] CqM11bt9QvI: Downloading m3u8 information
[info] CqM11bt9QvI: Downloading 1 format(s): 251
[download] Destination: tracce/The Libertines - Can't Stand Me Now (Official Video).webm
[download] 100% of    3.24MiB in 00:00:

[download] Got error: HTTPSConnectionPool(host='rr4---sn-hpa7kn76.googlevideo.com', port=443): Read timed out.


[download] Got error: HTTPSConnectionPool(host='rr4---sn-hpa7kn76.googlevideo.com', port=443): Read timed out.
[youtube] Extracting URL: https://www.youtube.com/watch?v=qkMUkZ0DjFo
[youtube] qkMUkZ0DjFo: Downloading webpage
[youtube] qkMUkZ0DjFo: Downloading ios player API JSON
[youtube] qkMUkZ0DjFo: Downloading android player API JSON
[youtube] qkMUkZ0DjFo: Downloading m3u8 information
[info] qkMUkZ0DjFo: Downloading 1 format(s): 251
[download] Destination: tracce/Broken Social Scene - Lover's Spit (Cover by Chris Rubeo from The Outs).webm
[download] 100% of    6.05MiB in 00:00:26 at 235.10KiB/s 
[ExtractAudio] Destination: tracce/Broken Social Scene - Lover's Spit (Cover by Chris Rubeo from The Outs).mp3
Deleting original file tracce/Broken Social Scene - Lover's Spit (Cover by Chris Rubeo from The Outs).webm (pass -k to keep)
Artista: Broken Social Scene, Traccia: Lover's Spit, Audio scaricato da soundcloud
Errore durante la ricerca di Spoon su Spotify: HTTPSConnectionPool(host='api

KeyboardInterrupt: 

In [11]:
import pandas as pd
dt_completo=pd.read_csv('/Users/danielnotaristefano/Desktop/Proggetto data mining/lastfm/items_new_mapping.tsv', sep='\t', header=None)
dt_filtrato=pd.read_csv('/Users/danielnotaristefano/Desktop/Proggetto data mining/lastfm/text/idtext_idgraph.tsv', sep='\t', header=None)
dt_dbpedia=pd.read_csv('/Users/danielnotaristefano/Desktop/Proggetto data mining/lastfm/items_original_mapping.txt', sep='\t', header=None)

In [51]:
#creazione dataset filtrato con nomi degli artisti presi da dt_completo
merged_df = dt_filtrato.merge(dt_completo, left_on=0, right_on=0)
del merged_df['1_x']



KeyError: '1_x'

In [52]:
#creazione file txt con carattere di tabulazione per il dataset filtrato per poterlo utilizzare negli script per scaricarel'audio e le copertine
merged_df.to_csv('merged_df.txt', sep='\t', index=False)


In [73]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

# Imposta le tue credenziali Spotify
client_id = '93193686a98745eda05ccb0c72ec7fee'
client_secret = '8eb7dd68e0f346a08abfa9ba41a6d12a'

# Crea un oggetto Spotipy
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Leggi il file di testo e estrai solo i nomi degli artisti 
with open('/Users/danielnotaristefano/Desktop/Proggetto data mining/lastfm/merged_df.txt', 'r') as file:
    artists_list = [line.strip().split('\t') for line in file]

# Lista per salvare gli URL delle copertine degli album
cover_urls = []

# Carica le copertine già scaricate
if os.path.exists('cover_urls.txt'):
    with open('cover_urls.txt', 'r') as url_file:
        cover_urls = [line.strip() for line in url_file]

# Creazione della cartella 'copertine'
os.makedirs('copertine', exist_ok=True)

# Scarica le copertine degli album per ciascun artista
for artist_info in artists_list:
    artist_id = artist_info[0]
    artist_name = artist_info[1]

    # Verifica se l'artista è già stato elaborato
    if any(artist_name in url for url in cover_urls):
        print(f"Artista già elaborato: {artist_name}")
        continue

    # Cerca gli album dell'artista su Spotify
    try:
        results = sp.search(q=f'artist:{artist_name}', type='album', limit=5)
    except requests.exceptions.Timeout:
        print(f"Timeout durante la ricerca su Spotify per l'artista: {artist_name}")
        continue

    # Scarica le copertine degli album
    for idx, album in enumerate(results['albums']['items']):
        album_title = album['name']
        cover_url = album['images'][0]['url'] if album['images'] else None

        if cover_url:
            # Aggiungi l'URL alle copertine alla lista
            cover_urls.append(f"{artist_name}\t{album_title}\t{cover_url}")

            # Scarica la copertina
            response = requests.get(cover_url)

            # Salva la copertina nella cartella 'copertine' con ID artista e numero sequenziale
            file_path = os.path.join('copertine', f"{artist_id}_{idx + 1}.jpg")
            with open(file_path, 'wb') as img_file:
                img_file.write(response.content)

            print(f"Artista: {artist_name}, Album: {album_title}, Copertina scaricata")
        else:
            print(f"Artista: {artist_name}, Album: {album_title}, Nessuna copertina disponibile")

# Salva gli URL delle copertine in un file di testo
with open('cover_urls.txt', 'w') as url_file:
    for url in cover_urls:
        url_file.write(url + '\n')


Artista: Diary of Dreams, Album: Grau im Licht, Copertina scaricata
Artista: Diary of Dreams, Album: The Anatomy of Silence, Copertina scaricata
Artista: Diary of Dreams, Album: Hell in Eden, Copertina scaricata
Artista: Diary of Dreams, Album: Melancholin, Copertina scaricata
Artista: Diary of Dreams, Album: Freak Perfume, Copertina scaricata
Artista: Moonspell, Album: Wolfheart, Copertina scaricata
Artista: Moonspell, Album: Night Eternal (Extended Version), Copertina scaricata
Artista: Moonspell, Album: Irreligious, Copertina scaricata
Artista: Moonspell, Album: Extinct (Deluxe Version), Copertina scaricata
Artista: Moonspell, Album: Memorial (Bonus Track Edition), Copertina scaricata
Artista: Marilyn Manson, Album: Antichrist Superstar, Copertina scaricata
Artista: Marilyn Manson, Album: Smells Like Children, Copertina scaricata
Artista: Marilyn Manson, Album: Holy Wood, Copertina scaricata
Artista: Marilyn Manson, Album: Lest We Forget - The Best Of, Copertina scaricata
Artista: M